# Fetch Snooker match data
Load multiple years worth of data for modelling and exploration purposes using the snooker.org API: [http://api.snooker.org/](http://api.snooker.org/)

### Imports

In [1]:
import requests
import json

import pandas as pd

pd.options.display.max_columns = 500

### Constants and utility functions

In [2]:
YEAR_START = 2005

YEAR_END = 2019

In [3]:
def fetch_players(year_start, year_end):
    players_json = []
    for year in range(year_start, year_end+1, 1):
        print('Fetching players from {}...'.format(year))
        players_json = players_json + requests.get('http://api.snooker.org/?t=10&st=p&s={}'.format(str(year))).json()
        
    return pd.DataFrame(players_json).drop_duplicates('ID')


def fetch_data(year_start, year_end, player_data=None):
    matches_json = []
    
    if player_data is None:
        player_data = fetch_players(year_start, year_end)
        
    for year in range(year_start, year_end+1, 1):
        for player_id in player_data['ID'].unique():
            try:
                matches_json = matches_json + requests.get('http://api.snooker.org/?t=8&p={}&s={}'.format(str(player_id), str(year))).json()
            except:
                continue
                
        print('Fetched match data for {}.'.format(year))
                
    return pd.DataFrame(matches_json).drop_duplicates()

def fetch_season_rankings(year_start, year_end):
    ranking_json = []
    for year in range(year_start-1, year_end, 1):  # season before the current event is what is required
        try:
            ranking_json = ranking_json + requests.get('http://api.snooker.org/?rt=MoneyRankings&s={}'.format(year)).json()
        except:
            continue
    
    return pd.DataFrame(ranking_json).drop_duplicates()


def fetch_events_data(year_start, year_end):
    events_json = []
    for year in range(year_start, year_end+1, 1):
        try:
            events_json = events_json + requests.get('http://api.snooker.org/?t=5&s={}'.format(year)).json()
        except:
            continue
    
    return pd.DataFrame(events_json).drop_duplicates()


def fetch_seedings_data(matches_data):
    seedings_json = []
    for event_id in matches['EventID'].unique():
        try:
            seedings_json = seedings_json + requests.get('http://api.snooker.org/?t=13&e={}'.format(event_id)).json()
        except:
            continue
    
    return pd.DataFrame(seedings_json).drop_duplicates()
    
    

### Data Fetching

In [4]:
players = fetch_players(YEAR_START, YEAR_END)

Fetching players from 2005...
Fetching players from 2006...
Fetching players from 2007...
Fetching players from 2008...
Fetching players from 2009...
Fetching players from 2010...
Fetching players from 2011...
Fetching players from 2012...
Fetching players from 2013...
Fetching players from 2014...
Fetching players from 2015...
Fetching players from 2016...
Fetching players from 2017...
Fetching players from 2018...
Fetching players from 2019...


In [5]:
players.head()

,BioPage,Born,Club,FirstName,FirstSeasonAsPro,ID,Info,LastName,LastSeasonAsPro,License,MiddleName,Nationality,Photo,PhotoSource,Sex,ShortName,SurnameFirst,TeamName,TeamNumber,TeamSeason,Twitter,Type,URL
0,http://snooker.org/plr/bio/mwilliams.shtml,1975-03-21,,Mark,1992,1,,Williams,0,,J,Wales,http://snooker.org/img/players/MarkWilliams.png,,M,M J Williams,False,,0,0,markwil147,1,
1,,1981-03-13,,Stephen,1998,2,,Maguire,0,,,Scotland,http://snooker.org/img/players/Maguire.png,,M,,False,,0,0,,1,
2,,1985-09-12,,Jamie,2002,3,,Cope,2016,,,England,http://snooker.org/img/players/JamieCope.png,,M,,False,,0,0,JamieCope147,1,
3,,1978-01-08,,Marco,1998,4,,Fu,0,,,Hong Kong,http://snooker.org/img/players/mfu.jpg,,M,,False,,0,0,Marcofu18,1,
4,http://snooker.org/plr/bio/rosullivan.shtml,1975-12-05,,Ronnie,1992,5,,O'Sullivan,0,,,England,http://snooker.org/img/players/rosullivan.jpg,,M,R O'Sullivan,False,,0,0,ronnieo147,1,


In [6]:
%%time
matches = fetch_data(YEAR_START, YEAR_END, players)

Fetched match data for 2005.
Fetched match data for 2006.
Fetched match data for 2007.
Fetched match data for 2008.
Fetched match data for 2009.
Fetched match data for 2010.
Fetched match data for 2011.
Fetched match data for 2012.
Fetched match data for 2013.
Fetched match data for 2014.
Fetched match data for 2015.
Fetched match data for 2016.
Fetched match data for 2017.
Fetched match data for 2018.
Fetched match data for 2019.
Wall time: 1h 53min 31s


In [7]:
matches.columns

Index(['DetailsUrl', 'EndDate', 'Estimated', 'EventID', 'ExtendedNote',
       'FrameScores', 'ID', 'InitDate', 'LiveUrl', 'ModDate', 'Note', 'Number',
       'OnBreak', 'Player1ID', 'Player2ID', 'PointsDropped', 'Round',
       'ScheduledDate', 'Score1', 'Score2', 'Sessions', 'ShowCommonNote',
       'StartDate', 'TableNo', 'Type', 'Unfinished', 'VideoURL', 'Walkover1',
       'Walkover2', 'WinnerID', 'WorldSnookerID'],
      dtype='object')

In [8]:
matches.shape

(35143, 31)

In [9]:
matches.head()

,DetailsUrl,EndDate,Estimated,EventID,ExtendedNote,FrameScores,ID,InitDate,LiveUrl,ModDate,Note,Number,OnBreak,Player1ID,Player2ID,PointsDropped,Round,ScheduledDate,Score1,Score2,Sessions,ShowCommonNote,StartDate,TableNo,Type,Unfinished,VideoURL,Walkover1,Walkover2,WinnerID,WorldSnookerID
0,,2013-04-20T23:53:43Z,False,300,,,1188157,2013-04-20T23:53:43Z,,2013-04-20T23:53:43Z,,14,False,1,115,False,7,,10,1,,False,2013-04-20T23:53:43Z,0,1,False,,False,False,1,0
1,,2013-04-20T23:58:20Z,False,300,,,1188166,2013-04-20T23:58:20Z,,2013-04-20T23:58:20Z,,7,False,17,1,False,8,,8,13,,False,2013-04-20T23:58:20Z,0,1,False,,False,False,1,0
2,,2013-04-21T00:01:27Z,False,300,,,1188171,2013-04-21T00:01:27Z,,2013-04-21T00:01:27Z,,4,False,1,5,False,13,,11,13,,False,2013-04-21T00:01:27Z,0,1,False,,False,False,5,0
3,,2020-06-30T20:57:57Z,False,1031,,,6029572,2020-06-30T20:57:57Z,,2020-06-30T20:57:57Z,,14,False,1,21,False,8,,5,3,,False,2020-06-30T20:57:57Z,0,1,False,,False,False,1,0
4,,2020-06-30T21:00:16Z,False,1031,,,6029593,2020-06-30T20:59:30Z,,2020-06-30T21:00:19Z,,7,False,9,1,False,9,,1,5,,False,2020-06-30T21:00:16Z,0,1,False,,False,False,1,0


In [10]:
matches['WIN'] = (matches['WinnerID'] == matches['Player1ID']).astype(int)

In [11]:
matches.to_csv('data/Matches_data.csv', index=False)

In [12]:
# matches = pd.read_csv('data/Matches_data.csv')

## Fetch and append other data sources

In [13]:
(~matches[['Walkover1', 'Walkover2']].any(axis=1)).value_counts()

True     34084
False     1059
dtype: int64

In [14]:
matches = matches.loc[~matches[['Walkover1', 'Walkover2']].any(axis=1)]

In [15]:
events = fetch_events_data(YEAR_START, YEAR_END)
seedings = fetch_seedings_data(matches)
rankings = fetch_season_rankings(YEAR_START, YEAR_END)

In [16]:
players.to_csv('data/Players_data.csv', index=False)
events.to_csv('data/Events_data.csv', index=False)
seedings.to_csv('data/Seedings_data.csv', index=False)
rankings.to_csv('data/Rankings_data.csv', index=False)

In [17]:
# players = pd.read_csv('data/Players_data.csv')
# events = pd.read_csv('data/Events_data.csv')
# seedings = pd.read_csv('data/Seedings_data.csv')
# rankings = pd.read_csv('data/Rankings_data.csv')

### Merge matches data with other sources for a comprehensive raw dataset

Merge events data.

In [18]:
matches = matches.merge(events.rename(columns={'ID': 'EventID'}), on='EventID', how='left', suffixes=('', '_event'))

Merge seedings data.

In [19]:
matches = matches.merge(
    seedings.rename(columns={'Seeding': 'Player1Seeding', 'PlayerID': 'Player1ID'}), 
    on=['EventID', 'Player1ID'], 
    how='left'
)
matches = matches.merge(
    seedings.rename(columns={'Seeding': 'Player2Seeding', 'PlayerID': 'Player2ID'}), 
    on=['EventID', 'Player2ID'], 
    how='left'
)

Merge player data.

In [20]:
matches = matches.merge(
    players[['ID', 'FirstSeasonAsPro', 'Nationality']].rename(
        columns={'ID': 'Player1ID', 'FirstSeasonAsPro': 'Player1FirstSeasonAsPro', 'Nationality': 'Player1Nationality'}
    ), 
    on='Player1ID', 
    how='left'
)
matches = matches.merge(
    players[['ID', 'FirstSeasonAsPro', 'Nationality']].rename(
        columns={'ID': 'Player2ID', 'FirstSeasonAsPro': 'Player2FirstSeasonAsPro', 'Nationality': 'Player2Nationality'}
    ), 
    on='Player2ID', 
    how='left'
)
matches['Player1YearsAsPro'] = matches['Season'] - matches['Player1FirstSeasonAsPro']
matches['Player2YearsAsPro'] = matches['Season'] - matches['Player2FirstSeasonAsPro']

In [21]:
matches['LastSeason'] = matches['Season'] - 1

Merge rankings data.

In [22]:
matches = matches.merge(
    rankings[['PlayerID', 'Position', 'Season', 'Sum']].rename(columns={
        'PlayerID': 'Player1ID', 'Position': 'Player1LastSeasonRank', 'Season': 'LastSeason', 'Sum': 'Player1LastSeasonSum'
    }),
    on=['Player1ID', 'LastSeason'],
    how='left'
)
matches = matches.merge(
    rankings[['PlayerID', 'Position', 'Season', 'Sum']].rename(columns={
        'PlayerID': 'Player2ID', 'Position': 'Player2LastSeasonRank', 'Season': 'LastSeason', 'Sum': 'Player2LastSeasonSum'
    }),
    on=['Player2ID', 'LastSeason'],
    how='left'
)

### Save

In [23]:
matches.shape

(34084, 82)

In [24]:
matches.head(3)

,DetailsUrl,EndDate,Estimated,EventID,ExtendedNote,FrameScores,ID,InitDate,LiveUrl,ModDate,Note,Number,OnBreak,Player1ID,Player2ID,PointsDropped,Round,ScheduledDate,Score1,Score2,Sessions,ShowCommonNote,StartDate,TableNo,Type,Unfinished,VideoURL,Walkover1,Walkover2,WinnerID,WorldSnookerID,WIN,AgeGroup,AllRoundsAdded,City,CommonNote,ConversionRate,Country,DefendingChampion,Discipline,EndDate_event,EventPredictionID,Format,HashTag,Main,Name,Note_event,Num,NumActive,NumCompetitors,NumResults,NumUpcoming,PhotoURLs,PreviousEdition,RankingType,Related,Season,Sex,ShortName,Sponsor,Stage,StartDate_event,Team,Twitter,Type_event,Url,ValueType,Venue,WorldSnookerId,Player1Seeding,Player2Seeding,Player1FirstSeasonAsPro,Player1Nationality,Player2FirstSeasonAsPro,Player2Nationality,Player1YearsAsPro,Player2YearsAsPro,LastSeason,Player1LastSeasonRank,Player1LastSeasonSum,Player2LastSeasonRank,Player2LastSeasonSum
0,,2013-04-20T23:53:43Z,False,300,,,1188157,2013-04-20T23:53:43Z,,2013-04-20T23:53:43Z,,14,False,1,115,False,7,,10,1,,False,2013-04-20T23:53:43Z,0,1,False,,False,False,1,0,1,O,True,Sheffield,,1.0,England,0,snooker,2006-05-01,0,1,,300,World Championship,,0,0,108,31,0,,0,WR,world,2005,Both,,888.com,F,2006-04-15,False,,Ranking,,WC,Crucible Theatre,0,NaN,NaN,1992.0,Wales,1991.0,England,13.0,14.0,2004,NaN,NaN,NaN,NaN
1,,2013-04-20T23:58:20Z,False,300,,,1188166,2013-04-20T23:58:20Z,,2013-04-20T23:58:20Z,,7,False,17,1,False,8,,8,13,,False,2013-04-20T23:58:20Z,0,1,False,,False,False,1,0,0,O,True,Sheffield,,1.0,England,0,snooker,2006-05-01,0,1,,300,World Championship,,0,0,108,31,0,,0,WR,world,2005,Both,,888.com,F,2006-04-15,False,,Ranking,,WC,Crucible Theatre,0,NaN,NaN,1999.0,England,1992.0,Wales,6.0,13.0,2004,NaN,NaN,NaN,NaN
2,,2013-04-21T00:01:27Z,False,300,,,1188171,2013-04-21T00:01:27Z,,2013-04-21T00:01:27Z,,4,False,1,5,False,13,,11,13,,False,2013-04-21T00:01:27Z,0,1,False,,False,False,5,0,0,O,True,Sheffield,,1.0,England,0,snooker,2006-05-01,0,1,,300,World Championship,,0,0,108,31,0,,0,WR,world,2005,Both,,888.com,F,2006-04-15,False,,Ranking,,WC,Crucible Theatre,0,NaN,NaN,1992.0,Wales,1992.0,England,13.0,13.0,2004,NaN,NaN,NaN,NaN


In [26]:
matches.columns

Index(['DetailsUrl', 'EndDate', 'Estimated', 'EventID', 'ExtendedNote',
       'FrameScores', 'ID', 'InitDate', 'LiveUrl', 'ModDate', 'Note', 'Number',
       'OnBreak', 'Player1ID', 'Player2ID', 'PointsDropped', 'Round',
       'ScheduledDate', 'Score1', 'Score2', 'Sessions', 'ShowCommonNote',
       'StartDate', 'TableNo', 'Type', 'Unfinished', 'VideoURL', 'Walkover1',
       'Walkover2', 'WinnerID', 'WorldSnookerID', 'WIN', 'AgeGroup',
       'AllRoundsAdded', 'City', 'CommonNote', 'ConversionRate', 'Country',
       'DefendingChampion', 'Discipline', 'EndDate_event', 'EventPredictionID',
       'Format', 'HashTag', 'Main', 'Name', 'Note_event', 'Num', 'NumActive',
       'NumCompetitors', 'NumResults', 'NumUpcoming', 'PhotoURLs',
       'PreviousEdition', 'RankingType', 'Related', 'Season', 'Sex',
       'ShortName', 'Sponsor', 'Stage', 'StartDate_event', 'Team', 'Twitter',
       'Type_event', 'Url', 'ValueType', 'Venue', 'WorldSnookerId',
       'Player1Seeding', 'Player2Seeding'

In [31]:
matches.to_csv('data/Raw_data.csv', index=False)